In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
import random

In [2]:
# Initialize Spark
spark = SparkSession.builder \
    .appName("Generate Employee CSVs") \
    .getOrCreate()

# Define output path
CSVpath = "/Users/kulyashdahiya/STUDY/DataEngineering/PysparkLearning/Basics/ELC_Try/TestFilesCreation/EmployeeCSVs/"

# Sample data to choose from
employee_names = ["John", "Alice", "Bob", "Carol", "David", "Eva", "Frank", "Grace", "Hank", "Ivy"]
departments = ["HR", "Engineering", "Finance", "Sales", "Marketing", "IT"]
manager_names = ["ManagerA", "ManagerB", "ManagerC", "ManagerD", "ManagerE"]

# Generate 100 files
for file_index in range(1, 101):
    num_records = random.randint(100, 200)
    data = []
    for i in range(num_records):
        employee_id = f"{file_index:03d}{i:04d}"
        employee_name = random.choice(employee_names)
        department = random.choice(departments)
        salary = round(random.uniform(40000, 120000), 2)
        manager = random.choice(manager_names)
        data.append((employee_id, employee_name, department, salary, manager))

    # Create DataFrame
    schema = StructType([
        StructField("EmployeeID", StringType(), True),
        StructField("EmployeeName", StringType(), True),
        StructField("DepartmentName", StringType(), True),
        StructField("Salary", DoubleType(), True),
        StructField("ManagerName", StringType(), True),
    ])

    df = spark.createDataFrame(data, schema)

    # Write each file individually
    output_file = f"{CSVpath}Employee_{file_index}.csv"
    df.coalesce(1).write.mode("overwrite").option("header", True).csv(output_file)

spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/25 15:40:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/25 15:40:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/25 15:40:14 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [9]:
import random, os
import shutil
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Save Single Parquet File") \
    .getOrCreate()

# Define output path
output_path = "/Users/kulyashdahiya/STUDY/DataEngineering/PysparkLearning/Basics/ELC_Try/TestFilesCreation/Singleparquet_file/Employees.parquet"
output_path = "/Users/kulyashdahiya/STUDY/DataEngineering/PysparkLearning/Basics/ELC_Try/TestFilesCreation/ParquetSave/Employee.parquet"

# Sample values
names = ["John", "Alice", "Bob", "Carol", "David", "Eva", "Frank", "Grace", "Hank", "Ivy"]
departments = ["HR", "Engineering", "Finance", "Sales", "Marketing", "IT"]
managers = ["ManagerA", "ManagerB", "ManagerC", "ManagerD", "ManagerE"]

# Generate 5000 records
data = []
for i in range(1, 5001):
    emp_id = f"E{i:05d}"
    name = random.choice(names)
    dept = random.choice(departments)
    salary = round(random.uniform(40000, 120000), 2)
    manager = random.choice(managers)
    data.append((emp_id, name, dept, salary, manager))

# Define schema
schema = StructType([
    StructField("EmployeeID", StringType(), True),
    StructField("EmployeeName", StringType(), True),
    StructField("DepartmentName", StringType(), True),
    StructField("Salary", DoubleType(), True),
    StructField("ManagerName", StringType(), True),
])

# Create DataFrame
df = spark.createDataFrame(data, schema)

# Save as a single Parquet file
# df.coalesce(1).write.mode("overwrite").parquet(output_path)
df.write.mode("overwrite").parquet(output_path)

# df.coalesce(1).write.mode("overwrite").parquet(output_path)

spark.stop()

25/04/25 19:43:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/25 19:43:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [8]:
# tempParquetFolder = "/Users/kulyashdahiya/STUDY/DataEngineering/PysparkLearning/Basics/ELC_Try/TestFilesCreation/Singleparquet_file_temp/"

output_path = "/Users/kulyashdahiya/STUDY/DataEngineering/PysparkLearning/Basics/ELC_Try/TestFilesCreation/Parquet/"

finalParquetFile = os.path.join(output_path, "Employee_12345.parquet")

tempParquetFile = "/Users/kulyashdahiya/STUDY/DataEngineering/PysparkLearning/Basics/ELC_Try/TestFilesCreation/Singleparquet_file/Employees.parquet/part-00000-95fa355e-cce9-44fb-b2a4-3cacf763bfba-c000.snappy.parquet"

shutil.move(tempParquetFile, finalParquetFile)

# Move the single Parquet part file to the final destination
# for temp_file in os.listdir(tempParquetFolder):
#     if temp_file.endswith(".parquet"):
#         shutil.move(os.path.join(tempParquetFolder, temp_file), finalParquetFile)
#         print(f"Moved {temp_file} to {finalParquetFile}")

# Clean up the temporary folder
# shutil.rmtree(tempParquetFolder)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kulyashdahiya/STUDY/DataEngineering/PysparkLearning/Basics/ELC_Try/TestFilesCreation/Parquet/Employee_12345.parquet'

In [ ]:
import os
import shutil

def processCSVAsset(self, preStagingFolder, batchID, dAsset, sFolder):
    """
    Processes a single CSV asset:
    - Reads CSV files from Pre-Staging.
    - Saves them as Parquet in the Staging folder.
    """
    inputPattern = os.path.join(preStagingFolder, dAsset["batchIngestFilePattern"])  # Path to CSV files
    self.aLogger.info(f"Preprocessing CSV for asset: {dAsset['assetName']} using pattern: {inputPattern}")

    # Construct the temporary Parquet save path
    tempParquetFolder = os.path.join(sFolder, f"{dAsset['assetName']}_{batchID}_temp")

    # Step 1: Read the CSV files into a DataFrame
    df = self.aSpark.read.option("header", "true").csv(inputPattern)

    # Step 2: Write the DataFrame as Parquet in the temporary folder
    self.aLogger.info(f"Saving Parquet temporarily for asset: {dAsset['assetName']} to {tempParquetFolder}")
    df.coalesce(1).write.mode("overwrite").parquet(tempParquetFolder)

    # Step 3: Move the Parquet file to the final destination and rename it
    tempParquetFile = os.path.join(tempParquetFolder, "part-00000-*.parquet")  # The part file inside the temporary folder
    finalParquetFile = os.path.join(sFolder, f"{dAsset['assetName']}_{batchID}.parquet")

    try:
        # Move the single Parquet part file to the final destination
        for temp_file in os.listdir(tempParquetFolder):
            if temp_file.endswith(".parquet"):
                shutil.move(os.path.join(tempParquetFolder, temp_file), finalParquetFile)
                self.aLogger.info(f"Moved {temp_file} to {finalParquetFile}")

        # Clean up the temporary folder
        shutil.rmtree(tempParquetFolder)

        # Update the asset with the new Parquet path
        dAsset["ingestFile"] = finalParquetFile
        self.aLogger.info(f"CSV asset {dAsset['assetName']} preprocessed and saved to {finalParquetFile}")

    except Exception as e:
        self.aLogger.error(f"Error moving Parquet file {tempParquetFile} to {finalParquetFile}: {str(e)}")
        raise

